In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/Colab Notebooks'
%ls

/content/drive/MyDrive/Colab Notebooks
 aischool                                    'Copy of tdcsfog_turn_test.csv'
'Copy of defog_start_hesitation.csv'         'Copy of tdcsfog_walking.csv'
'Copy of defog_start_hesitation_test.csv'    'Copy of tdcsfog_walking_test.csv'
'Copy of defog_turn.csv'                     'F23 HC7&8_analysis.ipynb'
'Copy of defog_turn_test.csv'                 MJFF_SVM_Model.ipynb
'Copy of defog_walking.csv'                  'My Copy of MJFF-FOG-Prediction-PD.ipynb'
'Copy of defog_walking_test.csv'              OldSVM.ipynb
'Copy of tdcsfog_start_hesitation.csv'        tlvmc-parkinsons-freezing-gait-prediction/
'Copy of tdcsfog_start_hesitation_test.csv'   tlvmc-parkinsons-freezing-gait-prediction.zip
'Copy of tdcsfog_turn.csv'


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix

In [5]:
#preprocessed training data
defog_turn = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Copy of defog_turn.csv')
defog_walk = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Copy of defog_walking.csv')
defog_sh = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Copy of defog_start_hesitation.csv')

tdcsfog_turn = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Copy of tdcsfog_turn.csv')
tdcsfog_walk = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Copy of tdcsfog_walking.csv')
tdcsfog_sh = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Copy of tdcsfog_start_hesitation.csv')

In [6]:
#preprocessed test data
defog_turn_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Copy of defog_turn_test.csv')
defog_walk_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Copy of defog_walking_test.csv')
defog_sh_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Copy of defog_start_hesitation_test.csv')

tdcsfog_turn_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Copy of tdcsfog_turn_test.csv')
tdcsfog_walk_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Copy of tdcsfog_walking_test.csv')
tdcsfog_sh_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Copy of tdcsfog_start_hesitation_test.csv')

In [7]:
#unnamed columns
defog_turn = defog_turn.drop(defog_turn.columns[0], axis=1)
defog_walk = defog_walk.drop(defog_walk.columns[0], axis=1)
defog_sh = defog_sh.drop(defog_sh.columns[0], axis=1)

tdcsfog_turn = tdcsfog_turn.drop(tdcsfog_turn.columns[0], axis=1)
tdcsfog_walk = tdcsfog_walk.drop(tdcsfog_walk.columns[0], axis=1)
tdcsfog_sh = tdcsfog_sh.drop(tdcsfog_sh.columns[0], axis=1)

defog_turn_test = defog_turn_test.drop(defog_turn_test.columns[0], axis=1)
defog_walk_test = defog_walk_test.drop(defog_walk_test.columns[0], axis=1)
defog_sh_test = defog_sh_test.drop(defog_sh_test.columns[0], axis=1)

tdcsfog_turn_test = tdcsfog_turn_test.drop(tdcsfog_turn_test.columns[0], axis=1)
tdcsfog_walk_test = tdcsfog_walk_test.drop(tdcsfog_walk_test.columns[0], axis=1)
tdcsfog_sh_test = tdcsfog_sh_test.drop(tdcsfog_sh_test.columns[0], axis=1)

In [8]:
X_train_dturn = defog_turn.drop(columns='Turn')
y_train_dturn = defog_turn['Turn']
############################
X_train_dwalk = defog_walk.drop(columns='Walking')
y_train_dwalk = defog_walk['Walking']
############################
X_train_dsh = defog_sh.drop(columns='StartHesitation')
y_train_dsh = defog_sh['StartHesitation']

#--------------------------#

X_train_tturn = tdcsfog_turn.drop(columns='Turn')
y_train_tturn = tdcsfog_turn['Turn']
############################
X_train_twalk = tdcsfog_walk.drop(columns='Walking')
y_train_twalk = tdcsfog_walk['Walking']
############################
X_train_tsh = tdcsfog_sh.drop(columns='StartHesitation')
y_train_tsh = tdcsfog_sh['StartHesitation']

#--------------------------#

X_test_dturn = defog_turn_test.drop(columns=['Turn'])
y_test_dturn = defog_turn_test['Turn']

X_test_dwalk = defog_walk_test.drop(columns=['Walking'])
y_test_dwalk = defog_walk_test['Walking']

X_test_dsh = defog_sh_test.drop(columns=['StartHesitation'])
y_test_dsh = defog_sh_test['StartHesitation']
############################
X_test_tturn = tdcsfog_turn_test.drop(columns=['Turn'])
y_test_tturn = tdcsfog_turn_test['Turn']

X_test_twalk = tdcsfog_walk_test.drop(columns=['Walking'])
y_test_twalk = tdcsfog_walk_test['Walking']

X_test_tsh = tdcsfog_sh_test.drop(columns=['StartHesitation'])
y_test_tsh = tdcsfog_sh_test['StartHesitation']

In [9]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [10]:
# Initialize SVM models for defog dataset
model_svc_dturn = SVC(class_weight='balanced')
model_svc_dwalk = SVC(class_weight='balanced')
model_svc_dsh = SVC(class_weight='balanced')

# Initialize SVM models for tdcsfog dataset
model_svc_tturn = SVC(class_weight='balanced')
model_svc_twalk = SVC(class_weight='balanced')
model_svc_tsh = SVC(class_weight='balanced')

In [11]:
%%time
model_svc_dturn.fit(X_train_dturn, y_train_dturn)
predicted_dturn = model_svc_dturn.predict(X_test_dturn)

CPU times: user 12.3 s, sys: 254 ms, total: 12.5 s
Wall time: 12.7 s


In [12]:
%%time
model_svc_dwalk.fit(X_train_dwalk, y_train_dwalk)
predicted_dwalk = model_svc_dwalk.predict(X_test_dwalk)

CPU times: user 13 s, sys: 59.1 ms, total: 13.1 s
Wall time: 13.1 s


In [13]:
%%time
model_svc_dsh.fit(X_train_dsh, y_train_dsh)
predicted_dsh = model_svc_dsh.predict(X_test_dsh)

CPU times: user 497 ms, sys: 1.8 ms, total: 499 ms
Wall time: 497 ms


In [14]:
%%time
model_svc_tturn.fit(X_train_tturn, y_train_tturn)
predicted_tturn = model_svc_tturn.predict(X_test_tturn)

CPU times: user 12.2 s, sys: 52 ms, total: 12.3 s
Wall time: 12.4 s


In [15]:
%%time
model_svc_twalk.fit(X_train_twalk, y_train_twalk)
predicted_twalk = model_svc_twalk.predict(X_test_twalk)

CPU times: user 28.7 s, sys: 67.5 ms, total: 28.8 s
Wall time: 29.3 s


In [16]:
%%time
model_svc_tsh.fit(X_train_tsh, y_train_tsh)
predicted_tsh = model_svc_tsh.predict(X_test_tsh)

CPU times: user 35.1 s, sys: 90.7 ms, total: 35.1 s
Wall time: 36 s


In [17]:
# Evaluate the models
from sklearn.metrics import classification_report

# Evaluate defog dataset
print("Defog - Turn Classification Report")
print(classification_report(y_test_dturn, predicted_dturn))

print("Defog - Walking Classification Report")
print(classification_report(y_test_dwalk, predicted_dwalk))

print("Defog - StartHesitation Classification Report")
print(classification_report(y_test_dsh, predicted_dsh))

# Evaluate tdcsfog dataset
print("TDCSFog - Turn Classification Report")
print(classification_report(y_test_tturn, predicted_tturn))

print("TDCSFog - Walking Classification Report")
print(classification_report(y_test_twalk, predicted_twalk))

print("TDCSFog - StartHesitation Classification Report")
print(classification_report(y_test_tsh, predicted_tsh))

Defog - Turn Classification Report
              precision    recall  f1-score   support

           0       0.99      0.49      0.66       813
           1       0.53      0.99      0.69       472

    accuracy                           0.68      1285
   macro avg       0.76      0.74      0.67      1285
weighted avg       0.82      0.68      0.67      1285

Defog - Walking Classification Report
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1256
           1       0.00      0.00      0.00        29

    accuracy                           0.98      1285
   macro avg       0.49      0.50      0.49      1285
weighted avg       0.96      0.98      0.97      1285

Defog - StartHesitation Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1285

    accuracy                           1.00      1285
   macro avg       1.00      1.00      1.00      1285
weighted 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [18]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced']
}

In [19]:
# Function for hyperparameter tuning and evaluation
def tune_and_train_svm(X_train, y_train, X_test, y_test, task_name):
    print(f"Tuning {task_name} model...\n")

    # Initialize GridSearchCV
    grid_search = GridSearchCV(
        estimator=SVC(),
        param_grid=param_grid,
        scoring='f1_macro',  # F1 score for imbalanced datasets
        cv=5,  # 5-fold cross-validation
        verbose=1,
        n_jobs=-1
    )

    # Fit the model
    grid_search.fit(X_train, y_train)

    # Best parameters
    print(f"Best parameters for {task_name}: {grid_search.best_params_}\n")

    # Predict with the best estimator
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(X_test)

    # Evaluate
    print(f"Classification Report for {task_name}:\n")
    print(classification_report(y_test, predictions))
    print("="*50)

    return best_model

In [ ]:
model_dturn = tune_and_train_svm(X_train_dturn, y_train_dturn, X_test_dturn, y_test_dturn, "Defog - Turn")
model_dwalk = tune_and_train_svm(X_train_dwalk, y_train_dwalk, X_test_dwalk, y_test_dwalk, "Defog - Walking")
model_dsh = tune_and_train_svm(X_train_dsh, y_train_dsh, X_test_dsh, y_test_dsh, "Defog - StartHesitation")

Tuning Defog - Turn model...

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best parameters for Defog - Turn: {'C': 100, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf'}

Classification Report for Defog - Turn:

              precision    recall  f1-score   support

           0       0.99      0.48      0.65       813
           1       0.53      0.99      0.69       472

    accuracy                           0.67      1285
   macro avg       0.76      0.74      0.67      1285
weighted avg       0.82      0.67      0.66      1285

Tuning Defog - Walking model...

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best parameters for Defog - Walking: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'linear'}

Classification Report for Defog - Walking:

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1256
           1       0.00      0.00      0.00        29

    accuracy  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best parameters for Defog - StartHesitation: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'linear'}

Classification Report for Defog - StartHesitation:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1285

    accuracy                           1.00      1285
   macro avg       1.00      1.00      1.00      1285
weighted avg       1.00      1.00      1.00      1285



In [ ]:
model_tturn = tune_and_train_svm(X_train_tturn, y_train_tturn, X_test_tturn, y_test_tturn, "TDCSFog - Turn")
model_twalk = tune_and_train_svm(X_train_twalk, y_train_twalk, X_test_twalk, y_test_twalk, "TDCSFog - Walking")
model_tsh = tune_and_train_svm(X_train_tsh, y_train_tsh, X_test_tsh, y_test_tsh, "TDCSFog - StartHesitation")

Tuning TDCSFog - Turn model...

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best parameters for TDCSFog - Turn: {'C': 100, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'rbf'}

Classification Report for TDCSFog - Turn:

              precision    recall  f1-score   support

           0       0.91      0.07      0.14       137
           1       0.64      1.00      0.78       224

    accuracy                           0.65       361
   macro avg       0.77      0.53      0.46       361
weighted avg       0.74      0.65      0.53       361

Tuning TDCSFog - Walking model...

Fitting 5 folds for each of 32 candidates, totalling 160 fits
